In [15]:
import cv2
import numpy as np
import os
import mediapipe as mp
from tqdm import tqdm  # Import tqdm for progress bar

In [16]:
mp_holistic = mp.solutions.holistic

In [17]:
# Function for Mediapipe Detection
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [18]:
# Function to Draw Styled Landmarks
def draw_styled_landmarks(image, results):
    mp_drawing = mp.solutions.drawing_utils
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


In [19]:
# Function to Extract Right Hand Keypoints
def extract_right_hand_keypoints(results):
    if results.right_hand_landmarks:
        keypoints = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark])
    else:
        keypoints = np.zeros((21, 3))  # 21 landmarks with (x, y, z)
    return keypoints.flatten()


In [20]:
# Define Paths and Parameters
actions_folder_path = "actions"
sequence_length = 30  # Number of frames per video

# Get all action subfolders
subfolders = [f for f in os.listdir(actions_folder_path) if os.path.isdir(os.path.join(actions_folder_path, f))]


In [21]:
# Process Videos
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action_folder in subfolders:
        action_path = os.path.join(actions_folder_path, action_folder)

        for video_file in tqdm(os.listdir(action_path), desc=f'Processing Action {action_folder}', unit='video'):
            video_path = os.path.join(action_path, video_file)
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            # Process only videos with exactly 30 frames
            if total_frames != sequence_length:
                print(f"Skipping video {video_file}: {total_frames} frames (not 30).")
                cap.release()
                continue

            frame_num = 0
            saved_frames = 0

            while cap.isOpened() and saved_frames < sequence_length:
                ret, frame = cap.read()
                if not ret:
                    break

                # Resize frame and process with Mediapipe
                frame = cv2.resize(frame, (640, 360))
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)

                # Extract and save right-hand keypoints
                keypoints = extract_right_hand_keypoints(results)
                npy_path = os.path.join(action_path, f"{os.path.splitext(video_file)[0]}_{saved_frames}.npy")
                np.save(npy_path, keypoints)

                saved_frames += 1

                # Display progress
                cv2.putText(image, f'Processing Action {action_folder}, Video {video_file}', 
                            (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

                # Exit on 'q' press
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

                frame_num += 1

            cap.release()

    cv2.destroyAllWindows()

Processing Action yes: 100%|██████████| 80/80 [03:29<00:00,  2.62s/video]


NameError: name 'subfolders' is not defined